In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Overwriting reducer.py


In [3]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [5]:
!hdfs dfs -mkdir /data/wiki/gaio_files

In [7]:
! hdfs dfs -ls /data/wiki/gaio_files

In [12]:
!hdfs dfs -ls /data/wiki/en_articles_part/articles-part/

-rwxrwxrwx   1 jovyan supergroup   76861985 2017-10-17 13:15 /data/wiki/en_articles_part/articles-part


In [14]:
!hdfs dfs -cat /data/wiki/en_articles_part/articles-part | head > a.txt

cat: Unable to write to output stream.


In [17]:
%ls

20180513-DemoNotebook.ipynb         DemoNotebook.ipynb  reducer.py
20180513-SMALLFILE-gaio-WORK.ipynb  mapper.py           supervisord.log
a.txt                               README.md           supervisord.pid


In [18]:
!hdfs dfs -copyFromLocal a.txt /data/wiki/gaio_files

In [19]:
! hdfs dfs -ls /data/wiki/gaio_files

Found 1 items
-rw-r--r--   1 jovyan supergroup     424525 2018-05-14 04:49 /data/wiki/gaio_files/a.txt


In [26]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/gaio_files \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

0	2
0-00-634011-3	1
0-19-824290-5	1
0-521-09456-9	1
0-7546-6196-2	1
0-8173-1065-7	2
0-8490-1397-6	1
0-85345-175-3	1
0.02	1
0.04	3


rm: `wordcount_result_1526295409276843': No such file or directory
18/05/14 10:56:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/05/14 10:56:55 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/05/14 10:56:56 INFO mapred.FileInputFormat: Total input files to process : 1
18/05/14 10:56:56 INFO mapreduce.JobSubmitter: number of splits:2
18/05/14 10:56:56 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1526272725029_0003
18/05/14 10:56:56 INFO impl.YarnClientImpl: Submitted application application_1526272725029_0003
18/05/14 10:56:56 INFO mapreduce.Job: The url to track the job: http://2bebfede4c70:8088/proxy/application_1526272725029_0003/
18/05/14 10:56:56 INFO mapreduce.Job: Running job: job_1526272725029_0003
18/05/14 10:57:09 INFO mapreduce.Job: Job job_1526272725029_0003 running in uber mode : false
18/05/14 10:57:09 INFO mapreduce.Job:  map 0% reduce 0%
18/05/14 10:57:26 INFO mapreduce.Job:  map 50% reduce 0%
18/05/14 10:5

In [28]:
!hdfs dfs -ls wordcount_result_1526295409276843

Found 2 items
-rw-r--r--   1 jovyan supergroup          0 2018-05-14 10:57 wordcount_result_1526295409276843/_SUCCESS
-rw-r--r--   1 jovyan supergroup     125310 2018-05-14 10:57 wordcount_result_1526295409276843/part-00000


In [29]:
!hdfs dfs -cat wordcount_result_1526295409276843/part-00000

0	2
0-00-634011-3	1
0-19-824290-5	1
0-521-09456-9	1
0-7546-6196-2	1
0-8173-1065-7	2
0-8490-1397-6	1
0-85345-175-3	1
0.02	1
0.04	3
0.05	1
0.08	1
0.09	2
0.1	3
0.12	2
0.15	4
0.17	1
0.18	2
0.2	1
0.25	1
0.3	1
0.4	2
0.40	1
0.5	1
0.55	1
0.5–0.7	1
0.6	2
0.7	2
0.8	2
0.80–0.90	1
0.9	5
0.96	1
0002-4341	1
1	28
1,000	8
1,128,047	1
1,200	1
1,376,957	1
1,380,121	1
1,541	1
1,800	1
1,866,452	1
1-4120-4843-5	1
1-55164-250-6	1
1-55164-251-4	1
1-60486-064-2	1
1-871082-16-1	1
1-d	1
1.1	1
1.5	2
1.58	1
10	18
10,000	2
100	3
100,000	4
1000	3
101,821	1
1025b25	1
104,430	1
104,991	1
105	1
106,280	1
108,000	1
10:00	1
10:13	1
10–15	1
11	15
11-volume	1
1100	1
111-113	1
112,249	1
113	1
116,250	1
118,572	1
119	1
119,490	1
12	11
12,280	1
12,410	1
12,770	1
12-year-old	1
120,000	1
120,155	1
121,054	1
121,110	1
121,800	1
123	1
126	1
12th	1
12–13	1
13	5
130,000	1
131–135	1
136	3
13th	1
14	7
14-v	1
14.3	1
140,247	2
143,000	1
145,639	1
147,137	1
14–34	1
15	13
150,647	1
150–151	1
153,829	1
154,040	1
1540	1
155	1
157th	1
15q1

distanced	1
distant	2
distinct	2
distinction	6
distinctly	2
distinguish	5
distinguishable	1
distinguished	7
distinguishes	2
distinguishing	1
distorting	1
distortion	1
distracted	1
distraction	1
distraught	1
distress	3
distributed	1
distributes	1
distribution	9
district	6
districts	2
distrust	1
disturb	1
disturbance	5
disturbances	1
disturbed	1
disunion	1
disused	1
dithyrambic	1
diverse	7
diversified	1
diversity	2
diversos	1
divide	1
divided	13
dividing	1
divine	5
divinity	1
divisible	1
division	2
divisions	1
divisiveness	1
divorcee	1
dixie	4
dixon	3
dna	1
do	35
do.the	1
doctor	4
doctoral	1
doctors	1
doctrine	7
doctrines	4
doctrines.phenomena	1
document	2
documentary	6
documentation	1
documented	1
documents	1
dodsworth	1
does	22
doesn’t	1
dogma	1
dogmatic	1
doing	4
dolce	1
dolgoff	2
dolittle	1
dollar	2
dollars	1
dolls	1
dolly	1
domestic	1
dominance	1
dominant	2
dominated	3
domination	2
dominici	1
dominion	1
domino	2
dominos	3
domitian	1
dommage	1
don	3
don't	1
donald	11
donate	1
donatin